In [79]:
import json
import random
import string
import os
import pandas as pd
import re

import nltk
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\adiha\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

## Building Model - Method 1

In [2]:
os.chdir("E:\\Documents\\Coding\\Notebooks\\Chatbot\\apotek")
print(os.getcwd())

E:\Documents\Coding\Notebooks\Chatbot\apotek


In [45]:
data_file = open("data.json").read()
data = json.loads(data_file)

In [46]:
words = []
classes = []
data_X = []
data_y = []

In [47]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens = nltk.word_tokenize(pattern)
        words.extend(tokens)
        data_X.append(pattern)
        data_y.append(intent["tag"])
    
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

In [48]:
# words = list(set(words))

In [49]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [50]:
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)

print(output)

ekonomi indonesia sedang dalam tumbuh yang bangga


In [51]:
words = [stemmer.stem(word.lower()) for word in words]
words = sorted(set(words))
classes = sorted(set(classes))

In [52]:
training = []
out_empty = [0] * len(classes)

In [53]:
for idx, doc in enumerate(data_X):
    bow = []
    text = stemmer.stem(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
        
    output_row = list(out_empty)
    output_row[classes.index(data_y[idx])] = 1
    
    training.append([bow, output_row])

In [54]:
random.shuffle(training)
training = np.array(training, dtype=object)

train_X = np.array(list(training[:,0]))
train_Y = np.array(list(training[:,1]))

In [55]:
print(train_Y[11])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [132]:
model = Sequential()
# model.add(Dense(256, input_shape=(len(train_X[0]),), activation="relu"))
# model.add(Dropout(0.5))
model.add(Dense(512, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
# model.add(Dense(32, activation='sigmoid'))
model.add(Dense(len(train_Y[0]), activation="softmax"))

adam = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=["accuracy"])

print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_60 (Dense)            (None, 512)               375808    
                                                                 
 dropout_40 (Dropout)        (None, 512)               0         
                                                                 
 dense_61 (Dense)            (None, 256)               131328    
                                                                 
 dropout_41 (Dropout)        (None, 256)               0         
                                                                 
 dense_62 (Dense)            (None, 429)               110253    
                                                                 
Total params: 617,389
Trainable params: 617,389
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
163/163 [=====================

In [180]:
def clean_text(text):
    tokens = nltk.word_tokenize(text)
    tokens = [stemmer.stem(word.lower()) for word in tokens if word in words]
    return tokens

In [16]:
def bag_of_words(text, vocab):
    tokens = clean_text(text)
    bow = [0] * len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word == w:
                bow[idx] = 1
    return np.array(bow)

In [74]:
def pred_class(text, vocab, labels):
    bow = bag_of_words(text, vocab)
    result = model.predict(np.array([bow]))[0]
    thresh = 0.5
    y_pred = [[indx, res] for indx, res in enumerate(result) if res > thresh]
    y_pred.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for r in y_pred:
        return_list.append(labels[r[0]])
    
    return return_list

In [18]:
def get_response(intents_list, intents_json):
    if len(intents_list) == 0:
        result = "Sorry! I don't understand"
    else:
        tag = intents_list[0]
        list_of_intents = intents_json["intents"]
        
        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["responses"])
                break
    return result

In [143]:
# model.save("model-2-full_product_name")

INFO:tensorflow:Assets written to: model-2-full_product_name\assets


## Testing Model

In [185]:
message = "erha truewhite"
intents = pred_class(message, words, classes)
result = get_response(intents, data)
# print(result)
y_pred = [[indx, res] for indx, res in enumerate(model.predict(np.array([bag_of_words(message, words)]))[0]) if res > 0.1]

for i in y_pred:
    print(classes[i[0]]," ",i[1])
print(intents)

1/1 [==============================] - 0s 54ms/step
erha_truewhite_face_toning   0.9999999
['erha_truewhite_face_toning']


In [32]:
print(classes)

['4d perfect nose & tip lift', 'abs', 'abs+bpo.25', 'abs+bpo.5', 'acl', 'acne guard cream', 'acne mask', 'acne_peeling', 'acneact bha & mugwort acne body wash 240 ml', 'acneact scar treatment by subcision rf', 'acneact_scar_treatment_by_microneedle_rf', 'acnic', 'acs1', 'acs2', 'acs3', 'acs4', 'acsbp', 'acsm', 'active acne therapy by ipl', 'advance home peeling', 'aef', 'af1', 'af2', 'af3', 'af4', 'af5', 'afp', 'aft', 'aftss', 'ag2', 'age corrector eye serum', 'age corrector firming body serum', 'age corrector nightcharge booster', 'age corrector serum', 'age corrector spot treatment', 'age reversal filler by aesthefill', 'ainf1', 'al1', 'al2', 'alcl3 .2', 'alcl3 10 alcohol', 'alcl3 10 aqua', 'all acs', 'all af', 'all al', 'all alcl', 'all amg', 'all anc', 'all aqua', 'all at', 'all aza', 'all bm', 'all carbamide', 'all cc', 'all clidacor', 'all cushion', 'all elo', 'all ga', 'all hiserha shampoo', 'all restore', 'all scalperfect shampoo', 'am', 'amg1', 'amg10', 'amg2', 'amg3', 'amg4',